<a href="https://colab.research.google.com/github/GEJ1/machine_learning_neurodiplo/blob/main/regression_and_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning resources

Author: Daniel Low (https://github.com/danielmlow)

Adapted from: https://github.com/danielmlow/tutorials/blob/main/machine_learning/regression_and_classification.ipynb

courses:
- http://materias.df.uba.ar/lda2021c1/171-2/
- https://www.kaggle.com/thirty-days-of-ml-assignments
- https://sebastianraschka.com/blog/2021/ml-course.html
- Coursera

Tutorials
- https://machinelearningmastery.com/machine-learning-in-python-step-by-step/
- Feature importance: https://machinelearningmastery.com/calculate-feature-importance-with-python/

Books:
- introductory book: Deuschle, WJ (2019). Undergraduate Fundamentals of Machine Learning link here. It's like a basic version of Murphy (see below)
- Advanced: Murphy, K. P. (2022). Machine learning: a probabilistic perspective. link
James et al (2021). An Introduction to Statistical Learning: with Applications in R (Springer Texts in Statistics)

Other
- Deep dive into evaluation metrics of performance and related analyses:  Varoquaux, G., & Colliot, O. (2022). Evaluating machine learning models and their diagnostic value.
- More resources (statistics, NLP): https://docs.google.com/document/d/127Npk6Z2gV-p_ewwnRz7qDyvKKRI6vb6Yg3zKnOw16s/edit?usp=sharing
- Machine learning without programming: https://jasp-stats.org/



# Machine learning tutorial

Data from: Torrente, F., Yoris, A., Low, D., Lopez, P., Bekinschtein, P., Vázquez, G. H., ... & Cetkovich, M. (2022). Psychological symptoms, mental fatigue and behavioural adherence after 72 continuous days of strict lockdown during the COVID-19 pandemic in Argentina. BJPsych open, 8(1).

Data were collected between 21 May 2020 and 4 June 2020 (early pandemic). The survey reached 3617 adults (85.2% female) from all provinces of Argentina after 72 days of lockdown. In that period, Argentina had an Oxford stringency index of 85/100. Of those surveyed, 45.6% met the cut-offs for depression (10 or higher), that is, PHQ-9  levels were much higher than at the start of the pandemic (33.7% [Torrente et al. (2021). J Affect Disord]) and pre-pandemic (5.6%; [Daray et al. (2017). J Affect Disord]) in Argentine samples.

Here we can try to predict a participant's depression score (either ordinal or binary) from their other mental health and demographic data.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from scipy.stats import pearsonr
from sklearn import metrics
from sklearn.linear_model import LassoCV, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
# setting to see all columns
pd.set_option('display.max_columns', None)

# figure size in inches
rcParams['figure.figsize'] = 5,5

# Load data and set input/output paths

In [ ]:
load_from_google_drive = False
# config: depends whether you're on Google Colab or local

# Get URL from github csv by clicking on Download > Copy Link Address
# https://github.com/danielmlow/covid19_argentina/blob/main/data/input/Emotional%20symptoms%20COVID19_Arg_May20_v2_text.csv
df = pd.read_csv('https://github.com/danielmlow/covid19_argentina/raw/main/data/input/Emotional%20symptoms%20COVID19_Arg_May20_v2_text.csv')


if load_from_google_drive:
  # On google colab
  # Mount GDrive and attach it to the colab for data I/O
  from google.colab import drive
  drive.mount('/content/drive')
  # input_dir = '/content/drive/My Drive/covid19_argentina/data/input/'
  output_dir = '/content/drive/My Drive/covid19_argentina/data/output/'
  os.makedirs(output_dir, exist_ok=True)

else:
  # If using jupyter-lab or jupyter notebook, load locally:
  input_dir = './data/input/'
  output_dir = './data/output/'



# Preprocessing

In [ ]:
# select subset of variables
df = df[[
    'PHQ9_Total',          # Puntuación total de depresión (PHQ-9)
    'GAD7_Total',          # Puntuación total de ansiedad (GAD-7)
    'Lockdown_adherence',  # Adherencia al confinamiento
    'Age',                 # Edad
    'Gender',              # Género
    'Family_Income',       # Ingreso familiar
    'Negat_thinking',      # Pensamiento negativo
    'COGN_DIFFICULTIES_index',
    'Mental_fatigue',
    'Threat',
    'Riskofcontagion',
    'Lockdown_difficulty',
    'Financialworry_present',
    'Financialworry_future',
    'Daily_stress_Index',
    'IU_total',            # Intolerancia a la incertidumbre
    'UCLA_LS_Total',       # Escala de soledad de UCLA
]]
df

In [ ]:
# Create dummy variable for categorical variable gender
gender = pd.get_dummies(df['Gender'],prefix='Gender')
gender = pd.get_dummies(gender,prefix='Gender')
df = df.drop('Gender', axis=1)
df['Gender_female'] = gender['Gender_1.0']
# other genders were not included since there were only 9.
variables = df.columns.values
variables

In [ ]:
# Use cleaner names in the final table

clean_names= dict(zip(
    variables,
    ['PHQ9', 'GAD7','Lockdown adherence','Age','Family income', 'Negative thinking','Cognitive troubles', 'Mental fatigue', 'Perceived threat', 'Perceived risk','Lockdown difficulty','Financial worries (present)','Financial worries (future)','Daily stress', 'Intolerance of uncertainty', 'Loneliness scale', 'Female', 'Male']

    ))

clean_names

# Descriptive statistics

In [ ]:
df.columns

In [ ]:
# Observe the descriptive stats of the independent variables
df.drop('PHQ9_Total',axis=1).describe()

### Take a subsample of observations and variables for faster plotting

In [ ]:
subsample = 0.1 # 0.1=10% of data, 1 = 100%
variables_to_plot = ['PHQ9_Total', 'GAD7_Total', 'Lockdown_adherence', 'Age','Family_Income'] # It
df_subsample = df[variables_to_plot].sample(frac=subsample)
sns.pairplot(df_subsample)
plt.show()

### More thorough stats

In [ ]:
from scipy.stats import spearmanr


def reg_coef(x,y,label=None,color=None, **kwargs):
    """Calcula y anota el coeficiente de correlación de Spearman y su significancia"""
    ax = plt.gca()
    r,p = spearmanr(x,y)
    if p < 0.01:
        sig_level = '***'
    elif p < 0.05:
        sig_level = '**'
    elif p < 0.05:
        sig_level = '*'
    else:
        sig_level = ''

    ax.annotate('r = {:.2f} {}'.format(r, sig_level), xy=(0.5,0.5), xycoords='axes fraction', ha='center')
    ax.texts[0].set_size(16)
    ax.set_axis_off()

# Create the plot
g = sns.PairGrid(data=df_subsample, vars=df_subsample.columns, hue=None)
g.map_upper(reg_coef)
g = g.map_lower(sns.regplot, scatter_kws={"edgecolor": "white"})
g = g.map_diag(sns.histplot, kde=True)
plt.show()

In [ ]:
def corrdot(*args, **kwargs):
    corr_r = args[0].corr(args[1], 'spearman')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca()
    ax.set_axis_off()
    marker_size = abs(corr_r) * 10000
    ax.scatter([.5], [.5], marker_size, [corr_r], alpha=0.75, cmap="coolwarm",
               vmin=-1, vmax=1, transform=ax.transAxes)
    font_size = 40 #abs(corr_r) * 40 + 5
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction",
                ha='center', va='center', fontsize=font_size)


# Pairwise correlation plot
sns.set(style='white', font_scale=1.6)
# iris = sns.load_dataset('iris')
g = sns.PairGrid(df_subsample, aspect=1.4, diag_sharey=False)
g.map_lower(sns.regplot, lowess=True, ci=True, line_kws={'color': 'black'}, fit_reg=True,
          x_jitter=.1, y_jitter=.1,
            scatter_kws={"s": 1, "alpha":0.1}
            )
g.map_diag(sns.histplot, kde_kws={'color': 'black'})
g.map_upper(corrdot)
plt.show()

# Create DFs for each independent variable

In [ ]:
def add_top_column(df, top_col, inplace=True):
    if not inplace:
        df = df.copy()

    df.columns = pd.MultiIndex.from_product([[top_col], df.columns])
    return df

In [ ]:
# Depression
variables_depression = df.drop('PHQ9_Total',axis=1).columns.values
X = df.drop('PHQ9_Total',axis=1).values
feature_names = df.drop('PHQ9_Total',axis=1).columns
feature_names = [clean_names.get(n) for n in feature_names]
y = df['PHQ9_Total'].values
print('depression independent variables', variables_depression)
print(X.shape, y.shape)
print('\n\n')

# # Anxiety

# variables_anxiety = df.drop('GAD7_Total',axis=1).columns.values
# X_anxiety = df.drop('GAD7_Total',axis=1).values
# y_anxiety = df['GAD7_Total'].values
# print('Anxiety independent variables', variables_anxiety)
# print(X_anxiety.shape, y_anxiety.shape)
# print('\n\n')


# # Lockdown Adherence

# variables_lockdown = df.drop('Lockdown_adherence',axis=1).columns.values
# X_lockdown = df.drop('Lockdown_adherence',axis=1).values
# y_lockdown = df['Lockdown_adherence'].values
# print('Lockdown Adherence independent variables', variables_lockdown)
# print(X_lockdown.shape, y_lockdown.shape)
# print('\n\n')



# Model evaluation

We're going to fit LASSO models and optimize the alpha or L1 parameter (regularization strength).

### Simple model


In [ ]:
from sklearn.model_selection import train_test_split

model_name_i = 'Lasso_80-20'

# Held-out cross-validation 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

model = Lasso()                # initializes model
model.fit(X_train,y_train)                             # fit model
intercept = model.intercept_                            # extract parameters
print('Intercept: ', np.round(intercept,4))
beta1 = model.coef_[0]                               # intercept and slopes same as in the .summary2() output above
y_pred = model.predict(X_test)
r2 = metrics.r2_score(y_test,y_pred)
print('Beta 1: ', np.round(beta1))
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False )
mae = metrics.mean_absolute_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
round_to = 2
results_dict = {
    'y_train_min': np.min(y_train),
    'y_train_max': np.max(y_train),
    'RMSE':np.round(rmse,round_to ),
    'MAE':np.round(mae,round_to ),
    'R^2':np.round(r2,round_to ),
    }
results = pd.DataFrame(results_dict, index=[model_name_i]).round(3)
results


In [ ]:
# Plot result for a regression task: true value vs predicted values
# ============================================================
plt.clf()
plt.scatter(y_test, y_pred)
plt.title(f"RMSE: {np.round(rmse,round_to )}\nMAE: {np.round(mae,round_to )}\nR^2: {np.round(r2,round_to )}")
plt.xlabel('True values')
plt.ylabel('Predicted values')
plt.tight_layout()
# plt.savefig(f'results_{ts}/scatter_{model_name_i}_{ts}.png', dpi=150)
plt.show()

In [ ]:
coefs = list(model.coef_)                     # Obtain coefficients from GridSearch
coefs= pd.DataFrame(coefs,index=feature_names, columns = ['Coef.']) # make DF
coefs['Abs. Coef.'] = coefs['Coef.'].abs()  # add column with absolute values to sort by, both positive and negative values are important.
coefs= coefs.sort_values('Abs. Coef.', ascending=False).reset_index() # sort by abs value and reset index to add a feature name column
coefs= coefs.drop(['Abs. Coef.'], axis=1)   # drop abs value, it's job is done
coefs.index +=1                             # Importance for publication, start index with 1 , as in 1st, 2nd, 3rd
coefs= coefs.reset_index()                  # turn index into column
coefs.columns= ['Importance', 'Feature', 'Std. Coef.'] # Clean column names
feature_importance = coefs.copy()
feature_importance

### k-fold cross-validation

In [ ]:
from sklearn.model_selection import KFold
splits = 5
kf = KFold(n_splits=splits, random_state=42, shuffle=True) # shuffle could be good if the data is ordered in same way (by ID, date) which could introduce bias.

scaler = StandardScaler()

y = np.array(y)                     # so I can index
results_mae = {}
results_r2 = {}
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

    model = Lasso()                # initializes model

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model.fit(X_train_scaled,y_train)                             # fit model

    y_pred = model.predict(X_test_scaled)
    rmse = metrics.mean_squared_error(y_test, y_pred, squared=False )
    mae = metrics.mean_absolute_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)
    round_to = 2
    results_dict = {
        f'MAE_{i}':np.round(mae,round_to ),
        f'R^2_{i}':np.round(r2,round_to ),
        }
    results_mae[f'MAE_{i}']=mae
    results_r2[f'R^2_{i}']=r2


In [ ]:
r2_all = pd.DataFrame(results_r2, index = ['score'])
r2_all['R^2'] = r2_all.mean(axis=1).round(3)
display(r2_all)
mae_all = pd.DataFrame(results_mae, index = ['score'])
mae_all['MAE'] = mae_all.mean(axis=1).round(3)
display(mae_all)


### Hyperparameter tuning on training set

In [ ]:
plt.rcParams["figure.figsize"] = (6,6)

In [ ]:
# Train and test
model_name_i = 'Lasso_80-20_tuned'

#80-20 Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning
alphas= np.logspace(-4, 0, 30) #equally spaced on log scale from 10**-4 (0.0004) to 10**0 (1)

# Pipeline: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', Lasso())
])

# hyperparameters and their values we're testing through gridsearch
param_grid = {
    'model__alpha': alphas,
}

# Do hyperparameter tuning within the training set, using 5-fold CV
search = GridSearchCV(pipe,param_grid, cv=5,n_jobs=-1) # perfrom k-fold CV on the training set

search.fit(X_train,y_train) # Train model
best_alpha = search.best_params_['model__alpha']
print('Best alpha: ', best_alpha)
# now use the best value to train the final model

# Train final model
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', Lasso(alpha = best_alpha))
])
pipe.fit(X_train,y_train) # train

# Test
y_pred = pipe.predict(X_test)

r2 = metrics.r2_score(y_test,y_pred)
print('Beta 1: ', np.round(beta1))
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False )
mae = metrics.mean_absolute_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
round_to = 2
results_dict = {
    'y_train_min': np.min(y_train),
    'y_train_max': np.max(y_train),
    'RMSE':np.round(rmse,round_to ),
    'MAE':np.round(mae,round_to ),
    'R^2':np.round(r2,round_to ),
    }
results = pd.DataFrame(results_dict, index=[model_name_i]).round(3)
display(results)


In [ ]:
# Plot result for a regression task: true value vs predicted values
# ============================================================
plt.clf()
plt.scatter(y_test, y_pred)
plt.title(f"RMSE: {np.round(rmse,round_to )}\nMAE: {np.round(mae,round_to )}\nR^2: {np.round(r2,round_to )}"
        )
plt.xlabel('True values')
plt.ylabel('Predicted values')
plt.tight_layout()
# plt.savefig(f'results_{ts}/scatter_{model_name_i}_{ts}.png', dpi=150)
plt.show()


In [ ]:
coefs = list(model.coef_)                     # Obtain coefficients from GridSearch
coefs= pd.DataFrame(coefs,index=feature_names, columns = ['Coef.']) # make DF
coefs['Abs. Coef.'] = coefs['Coef.'].abs()  # add column with absolute values to sort by, both positive and negative values are important.
coefs= coefs.sort_values('Abs. Coef.', ascending=False).reset_index() # sort by abs value and reset index to add a feature name column
coefs= coefs.drop(['Abs. Coef.'], axis=1)   # drop abs value, it's job is done
coefs.index +=1                             # Importance for publication, start index with 1 , as in 1st, 2nd, 3rd
coefs= coefs.reset_index()                  # turn index into column
coefs.columns= ['Importance', 'Feature', 'Std. Coef.'] # Clean column names
feature_importance = coefs.copy()
feature_importance

# Classification

In [ ]:
plt.rcParams["figure.figsize"] = (4,4)

In [ ]:
import random
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import f1_score, roc_auc_score
from matplotlib.pyplot import figure
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# plt.figure(figsize=(4,4))
plt.hist(y)
plt.xlabel('PHQ9')
plt.ylabel('Count')
plt.show()

In [ ]:
y_depression_binary = []
for i in y:
  if i >= 10:
    y_depression_binary.append(1)
  else:
    y_depression_binary.append(0)

y_depression_binary = np.array(y_depression_binary)
plt.hist(y_depression_binary)
plt.xticks(ticks = [0,1], labels = ['Not depressed', 'Depressed'])
plt.ylabel('Count')
plt.show()

In [ ]:
Counter(y_depression_binary) # Desbalanceado

In [ ]:
np.round(alphas,3)

In [ ]:
params_grid = [
              {
                'estimator':[LogisticRegression(class_weight='balanced', penalty = 'l1')],
                'estimator__C': [0.1, 0.3, 0.6, 0.9],
                },
                {
                'estimator':[SVC(class_weight='balanced',probability=True)],
                'estimator__C': [0.1, 1, 10],
                 'estimator__kernel': ['linear', 'rbf'],
                },
               # For the ROC AUC below, we need to output the probabilities
                # {
                # 'estimator': [RandomForestClassifier()],
                # 'estimator__n_estimators': [50, 250],
                # 'estimator__max_depth': [3,10],
                # },
               ]

In [ ]:
%%time
toy = False

if toy:
  # use a subsample
  m = 1000
  X_toy, y_depression_binary_toy = zip(*random.sample(list(zip(X, y_depression_binary)), m))
  X_train, X_test, y_train, y_test = train_test_split(X_toy, y_depression_binary_toy, test_size=0.2, random_state=42)
else:
  X_train, X_test, y_train, y_test = train_test_split(X, y_depression_binary, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('normalize', StandardScaler()),
    ('estimator', LogisticRegression(class_weight='balanced'))
    ])#initialize with any classifier, but then it will test all estimators specified in params_grid

# Train the grid search model, use refit, to refit the best params, scoring is what you want to optimize
gs = GridSearchCV(pipeline, params_grid, cv=5, n_jobs=-1, scoring='f1', refit=True).fit(X_train, y_train) # I use f1-score because it is less affected by class imbalance
# Best performing model and its corresponding hyperparameters
print('=====')
print('best parameters', gs.best_params_)
print('=====')

# Test data performance
y_proba = gs.predict_proba(X_test)       # Get predicted probabilities
y_proba = np.round(y_proba,2)               # round to 2 decimals
y_proba                                     # probabilitiesfor y=0 and y=1
y_proba_1 = y_proba[:,1]
y_pred = np.argmax(y_proba, axis=1) # in binary classification, threshold 0.5

print(classification_report(y_pred, y_test))
cm = confusion_matrix(y_test, y_pred, )

cm_display = ConfusionMatrixDisplay(cm, display_labels = ['Healthy', 'Depressed']).plot()
plt.grid(False)


In [ ]:

# Custom classification report
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
specificity = metrics.recall_score(y_test, y_pred, pos_label=0)    # specificity is the recall of the negative class or control group
f1 = metrics.f1_score(y_test, y_pred)
roc_auc = metrics.roc_auc_score(y_test, y_proba_1)  # IMPORTANT: other metrics take binary predictions y_pred. Here we test different thresholds, so we need probabilities (this will change the ROC AUC score)

results_dict = {
    'Precision':precision,
    'Recall':recall,
    'Specificity':specificity,
    'F1':f1,
    'ROC AUC':roc_auc,
    }

results = pd.DataFrame(results_dict, index=['model']).round(3)
results['best_params'] =  str(gs.best_params_)
results

### Classification permutation import

In [ ]:
# https://scikit-learn.org/1.5/modules/permutation_importance.html

result = permutation_importance(gs, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=-1)
figure(figsize=(12,8), dpi=300)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
fig.set_size_inches(12,8)
ax.boxplot(result.importances[sorted_idx].T,
           vert=False, labels=variables_depression[sorted_idx])
ax.set_title("Permutation Importances (test set)")
fig.tight_layout()
plt.show()